# PyTorch Hyperparameter Tuning (Skorch + GridSearchCV)



In [ ]:
import json
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from skorch import NeuralNetClassifier

data_dir = Path("../../data/features")

In [ ]:
def load_features(features_dir: Path = data_dir):
    suffix = "_v2" if (features_dir/"train_features_v2.X.npy").exists() else ""
    x_train = np.load(features_dir/f"train_features{suffix}.X.npy")
    y_train = np.load(features_dir/f"train_features{suffix}.y.npy")
    x_val = np.load(features_dir/f"val_features{suffix}.X.npy")
    y_val = np.load(features_dir/f"val_features{suffix}.y.npy")
    x_test = np.load(features_dir / f"test_features{suffix}.X.npy")
    y_test = np.load(features_dir / f"test_features{suffix}.y.npy")
    names_path = features_dir / f"feature_names{suffix}.json"
    if not names_path.exists():
        names_path = features_dir/"feature_names.json"
    with names_path.open() as f:
        feature_names = json.load(f)
    return x_train, y_train, x_val, y_val, x_test, y_test, feature_names


x_train, y_train, x_val, y_val, x_test, y_test, feature_names = load_features()
x_cv = np.vstack([x_train, x_val]).astype(np.float32)
y_cv = np.concatenate([y_train, y_val]).astype(np.int64)



In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim: int, num_neurons: int = 64, dropout_rate: float = 0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, num_neurons),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(num_neurons, 2),
        )

    def forward(self, x):
        return self.net(x)


def build_net(input_dim: int):
    return NeuralNetClassifier(
        module=SimpleNN,
        module__input_dim=input_dim,
        criterion=nn.CrossEntropyLoss,
        optimizer=optim.Adam,
        max_epochs=20,
        lr=0.001,
        batch_size=256,
        iterator_train__shuffle=True,
        verbose=0,
        device="cuda" if torch.cuda.is_available() else "cpu",
    )


net = build_net(x_cv.shape[1])



In [ ]:
param_grid = {
    "module__num_neurons": [64, 128, 256],
    "module__dropout_rate": [0.2, 0.3, 0.4],
    "lr": [0.0005, 0.001, 0.002],
    "max_epochs": [20, 40],
}

grid_search = GridSearchCV(
    estimator=net,
    param_grid=param_grid,
    cv=3,
    scoring="roc_auc",
    verbose=1,
    n_jobs=-1,
)

grid_search.fit(x_cv, y_cv)

In [ ]:
best_model = grid_search.best_estimator_
x_test_f = x_test.astype(np.float32)
y_test_pred = best_model.predict(x_test_f)
test_acc = accuracy_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, best_model.predict_proba(x_test_f)[:, 1])

print("Best Parameters", grid_search.best_params_)
print("Best CV ROC_AUC", round(grid_search.best_score_, 4))
print("Test Accuracy", round(test_acc, 4))
print("Test ROC_AUC", round(test_auc, 4))